<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/whisper_v3_20231106.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai-whisper==20231106

In [ ]:
# prompt: whisperをpythonで書く
import whisper

# whisper v3をインストール

In [ ]:
model = whisper.load_model("large-v3")

100%|█████████████████████████████████████| 2.88G/2.88G [01:14<00:00, 41.5MiB/s]


# Google Driveにアクセスして動画ファイル(のパスを)取得

In [ ]:
# prompt: google driveにアクセスして動画ファイルを取得
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir("/content/drive/MyDrive/video/")

['test.mp4', 'transcribe (26).srt']

In [ ]:
path_video = "/content/drive/MyDrive/video/test.mp4"

# 文字起こしを行う  

In [ ]:
result = model.transcribe(path_video)

In [ ]:
import pandas as pd

In [ ]:
df_v3_transcribe = pd.DataFrame(result["segments"])

In [ ]:
df_v3_transcribe.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,3.86,このアシスタントAPIを使うには最初にまずアシスタントというのを作ります,"[50365, 13303, 12817, 11054, 9550, 12144, 2997...",0.0,-0.113372,1.69186,0.033448
1,1,0,3.86,6.80,オリジナルのGPTが作れますよという感じですね,"[50558, 18743, 12376, 14655, 26719, 9405, 2972...",0.0,-0.113372,1.69186,0.033448
2,2,0,6.80,8.86,皆さんこんにちは、AI部長にゃんたです,"[50705, 41574, 38088, 1231, 48698, 13470, 1535...",0.0,-0.113372,1.69186,0.033448
3,3,0,8.86,11.64,本日はオープンAIデブデイということで,"[50808, 8802, 6890, 3065, 18743, 15266, 245, 4...",0.0,-0.113372,1.69186,0.033448
4,4,0,11.64,14.44,オープンAIから新しい機能がたくさん出てきたので,"[50947, 18743, 15266, 245, 4824, 48698, 11046,...",0.0,-0.113372,1.69186,0.033448


In [ ]:
def create_file(file_name="transcribe", results=None, file_type="srt"):
    if file_type not in ["srt", "txt"]:
        raise ValueError("ファイル拡張子は[srt, txt]を選択してください。")
    with open(f"{file_name}.{file_type}", mode="w", encoding='utf-8') as f:
        for index, _dict in enumerate(results):
            start_time, end_time, text = _dict["start"], _dict["end"], _dict["text"]

            # 時、分、秒、ミリ秒に分割
            s_h, s_m, s_s = int(start_time // 3600), int((start_time % 3600) // 60), int(start_time % 60)
            e_h, e_m, e_s = int(end_time // 3600), int((end_time % 3600) // 60), int(end_time % 60)

            # ミリ秒を計算
            s_ms = int((start_time - int(start_time)) * 1000)
            e_ms = int((end_time - int(end_time)) * 1000)

            # ファイルタイプに応じたフォーマットで出力
            if file_type == "srt":
                f.write(f"{index+1}\n")
                f.write(f"{s_h:02}:{s_m:02}:{s_s:02},{s_ms:03} --> {e_h:02}:{e_m:02}:{e_s:02},{e_ms:03}\n")
                f.write(f"{text}\n\n")
            elif file_type == "txt":
                f.write(f"[{s_h:02}:{s_m:02}:{s_s:02},{s_ms:03} ===> {e_h:02}:{e_m:02}:{e_s:02},{e_ms:03}] {text}\n")

In [ ]:
create_file(file_name="transcribe",results=result["segments"], file_type="srt")

In [ ]:
create_file(file_name="transcribe",results=result["segments"], file_type="txt")

# 英語に翻訳して文字起こしを行う

In [ ]:
result_translate = model.transcribe(audio=path_video, task="translate", language="en")

In [ ]:
df_v3_translate = pd.DataFrame(result_translate["segments"]).head()

In [ ]:
create_file(file_name="translate",results=result_translate["segments"], file_type="srt")

In [ ]:
create_file(file_name="translate",results=result_translate["segments"], file_type="txt")

# whisper-v2と比較してどうか？

In [ ]:
import torch
import gc

# Whisperモデルを削除
del model

# ガベージコレクタを使用してメモリを解放
gc.collect()

# PyTorchのキャッシュされたメモリを解放
torch.cuda.empty_cache()

In [ ]:
model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 2.87G/2.87G [00:46<00:00, 67.0MiB/s]


In [ ]:
result_v2 = model.transcribe(path_video)

In [ ]:
df_v2_transcribe = pd.DataFrame(result_v2["segments"])

In [ ]:
df_merge = pd.merge(df_v2_transcribe[["id", "start", "end", "text"]],
                     df_v3_transcribe[["id", "start", "end", "text"]],
                     how="outer", on="id", suffixes=('_v2', '_v3'))

df_merge["in_v2"] = df_merge["text_v2"].isin(df_merge["text_v3"].tolist())
df_merge["in_v3"] = df_merge["text_v3"].isin(df_merge["text_v2"].tolist())

In [ ]:
df_merge[((df_merge["in_v2"]==False)|(df_merge["in_v3"]==False))&(~df_merge["text_v2"].isnull())][["text_v2", "text_v3"]]

,text_v2,text_v3
0,このアシスタントAPI使うには最初にまずアシスタントというのを作ります,このアシスタントAPIを使うには最初にまずアシスタントというのを作ります
1,オリジナルのGPTが作れますよっていうふうな感じですね,オリジナルのGPTが作れますよという感じですね
2,皆さんこんにちはAI部長にゃんたんです,皆さんこんにちは、AI部長にゃんたです
4,オープンAIから新しい機能がたくさん出てきたのでそちらについてまとめていきます,オープンAIから新しい機能がたくさん出てきたので
5,日本時間の11月7日の午前3時からデベロッパーズカンファレンスというのが開催されてたんですが,そちらについてまとめていきます
6,皆さん見てみましたか,日本時間の11月7日の午前3時から
7,午前3時からだったのでにゃんたんはですね,Developers Conferenceというのが開催されていたんですが
8,前日の夜9時に寝てリアルタイムで見てみようと思ったんですけども,皆さん見てみましたか?
9,ちょっと寝過ごしてしまって朝5時に起きて今まとめています,午前3時からだったのでにゃんたはですね
10,こちらのカンファレンスに関してはYouTubeでも見れるので,前日の夜9時に寝てリアルタイムで見てみようと思ったんですけども
